In [1]:
import pandas as pd
from datetime import datetime, date, timedelta
from time import mktime
from time import strptime, strftime
import glob
import os

import netCDF4
import xarray as xr
import numpy as np
import re

import collections
import itertools


This code is creating a dictionary called period_dict and populating it with data from a range of dates for the given directory. The directory and directory_str variables are set to specific paths. The start and end Variables are set to store pd.Timestamp objects initialized with the start and end dates. day_range is then populated by using the pd.date_range function, which creates a range of dates incrementing in days between the start and end variables.

For each day in the day_range, this code will find the previous_step and next_step which sets the day time range between 3 hours prior and 1 day after each given date. range is then populated with a date range between the previous_step and next_step variables, with an increment of 3 hours. The filename_template contains a string with placeholders where t (from the range) and directory_str can be added to create a filename. This filename_template is then used to create a complete filename with the format function, which populates ind_period with the correct filename for each date and adds it to period_dict.

In [18]:
directory = 'Z:\\wse\\'
directory_str = 'Z:\\wse\\wse_500m_'
start = pd.Timestamp('2022-09-01')
end = pd.Timestamp('2022-09-14')
# Find date range of days in the given range 
day_range = pd.date_range(start=start, end=end, freq="D")

# Create period_dict to store the output data 
period_dict = {}

# Loop over the day_range
for d in day_range:
 
    # Set previous step to 3 hour earlier from the d
    previous_step = d - timedelta(hours=3)
    
    # Set next step to 1 day from the d
    next_step = d + timedelta(days=1)
    
    # Initialize dictionary with string version of 'd' as key
    period_dict[d.strftime('y%Y_d%j')] = []
    
    # Find date range of the hours in between previous_step and next_step with the frequency of 3 hours
    range = pd.date_range(start=previous_step, end=next_step, freq='3H')

    # Loop over the range 
    for t in range:
        # Make filename template by filling in the required data
        stamp = t.strftime('%Y%m%dT%H%M')
        filename_template = "{directory_str}{stamp}_y{t.year}_d{t.day_of_year}.nc"
        
        # Execute format on the filename_template 
        ind_period = filename_template.format(directory_str=directory_str, stamp=stamp, t=t)
        
        # Store the ind_period in the period_dict with the selected key
        period_dict[d.strftime('y%Y_d%j')].append(ind_period)



In [20]:
period_dict

{'y2022_d244': ['Z:\\wse\\wse_500m_20220831T2100_y2022_d243.nc',
  'Z:\\wse\\wse_500m_20220901T0000_y2022_d244.nc',
  'Z:\\wse\\wse_500m_20220901T0300_y2022_d244.nc',
  'Z:\\wse\\wse_500m_20220901T0600_y2022_d244.nc',
  'Z:\\wse\\wse_500m_20220901T0900_y2022_d244.nc',
  'Z:\\wse\\wse_500m_20220901T1200_y2022_d244.nc',
  'Z:\\wse\\wse_500m_20220901T1500_y2022_d244.nc',
  'Z:\\wse\\wse_500m_20220901T1800_y2022_d244.nc',
  'Z:\\wse\\wse_500m_20220901T2100_y2022_d244.nc',
  'Z:\\wse\\wse_500m_20220902T0000_y2022_d245.nc'],
 'y2022_d245': ['Z:\\wse\\wse_500m_20220901T2100_y2022_d244.nc',
  'Z:\\wse\\wse_500m_20220902T0000_y2022_d245.nc',
  'Z:\\wse\\wse_500m_20220902T0300_y2022_d245.nc',
  'Z:\\wse\\wse_500m_20220902T0600_y2022_d245.nc',
  'Z:\\wse\\wse_500m_20220902T0900_y2022_d245.nc',
  'Z:\\wse\\wse_500m_20220902T1200_y2022_d245.nc',
  'Z:\\wse\\wse_500m_20220902T1500_y2022_d245.nc',
  'Z:\\wse\\wse_500m_20220902T1800_y2022_d245.nc',
  'Z:\\wse\\wse_500m_20220902T2100_y2022_d245.nc',
  

In [19]:
print(t.day_of_year)

258


In [12]:
# Daylight Savings Time
directory = 'Z:\\wse\\'
directory_str = 'Z:\\wse\\wse_500m_'
start = pd.Timestamp('2022-09-01')
end = pd.Timestamp('2022-09-14')
day_range = pd.date_range(start=start,end=end,freq="D")
day_list = []
period_dict = {}

for d in day_range:
    day_list.append('y' + d.strftime('%Y') + '_d' + d.strftime('%j'))

for dd in day_list:
    previous_step = (pd.Timestamp((mktime(strptime(dd,'y%Y_d%j'))-18000)*1000000000)) - timedelta(hours=3)
    next_step = (pd.Timestamp((mktime(strptime(dd,'y%Y_d%j'))-18000)*1000000000)) + timedelta(days=1)
    period_dict[dd] = []
    range = pd.date_range(start=previous_step, end=next_step, freq='3H')
    for t in range:
        ind_period = str(directory_str + t.strftime('%Y%m%dT%H%M') +'_y' + t.strftime('%Y') + '_d' + t.strftime('%j') +'.nc')
        period_dict[dd].append(ind_period)


In [76]:
# Standard Time
directory = 'Z:\\wse\\'
directory_str = 'Z:\\wse\\wse_500m_'
start = pd.Timestamp('2021-12-01')
end = pd.Timestamp('2022-03-13')
day_range = pd.date_range(start=start,end=end,freq="D")
day_list = []
period_dict = {}

for d in day_range:
    day_list.append('y' + d.strftime('%Y') + '_d' + d.strftime('%j'))

for dd in day_list:
    previous_step = (pd.Timestamp((mktime(strptime(dd,'y%Y_d%j'))-21600)*1000000000)) - timedelta(hours=3)
    next_step = (pd.Timestamp((mktime(strptime(dd,'y%Y_d%j'))-21600)*1000000000)) + timedelta(days=1)
    period_dict[dd] = []
    range = pd.date_range(start=previous_step, end=next_step, freq='3H')
    for t in range:
        ind_period = str(directory_str + t.strftime('%Y%m%dT%H%M') +'_y' + t.strftime('%Y') + '_d' + t.strftime('%j') +'.nc')
        period_dict[dd].append(ind_period)


In [7]:
for i in day_list:
    ds = xr.open_mfdataset(period_dict[str(i)], combine = 'by_coords')
    t = pd.to_datetime(ds['time'].values[2]).strftime('%Y%m%d')
    output_name = f'wse_500m_{t}_combine.nc'
    ds.to_netcdf(directory + output_name)

In [ ]:
'''
lookup_expand = lookup
for i in merge_list:
    if i != merge_list[0]:
        
        first_val_backward = (list(lookup.items())[merge_list.index(i) - 1][1][-2])
        lookup_expand[i].insert(0, first_val_backward) 

    if i != merge_list[-1]:

        first_val_forward = (list(lookup.items())[merge_list.index(i) + 1][1][0])
        lookup_expand[i].append(first_val_forward)
'''